In [1]:
import numpy as np
from keras import callbacks
from keras.callbacks import History
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import utils
import string
from typing import List, Tuple
import re

In [2]:
import pandas as pd
from pandas import DataFrame


def load_lyrics(csv_path: str) -> DataFrame:
    df = pd.read_csv(csv_path, sep='\n', header=None)
    res = df.iloc[:, 0].str.rstrip(r'&, ').str.extract(r'([^,]+),([^,]+),(.+)')
    res.columns = ['artist', 'title', 'lyrics']
    return res


In [3]:
def create_x_y_train(songs, tokenize, total_words) -> Tuple[np.ndarray, np.ndarray]:
    input_sequences = []
    for line in songs:
        token_list = tokenize.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)
    print(input_sequences[:10])
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=100, padding='pre'))
    x_train = input_sequences[:, :-1]
    y_train = input_sequences[:, -1]
    y_train = utils.to_categorical(y_train, num_classes=total_words)
    return x_train, y_train, max_sequence_len

import pandas as pd
from pandas import DataFrame


def load_lyrics(csv_path: str) -> DataFrame:
    df = pd.read_csv(csv_path, sep='\n', header=None)
    res = df.iloc[:, 0].str.rstrip(r'&, ').str.extract(r'([^,]+),([^,]+),(.+)')
    res.columns = ['artist', 'title', 'lyrics']
    return res

def load_songs(path) -> List[str]:
    df = load_lyrics(path)
    songs = []
    for song in list(df['lyrics']):
        song += " EOF"
        # remove '(*)'
        modified_song = re.sub(r"\([^()]*\)", "", song)
        modified_song = modified_song.replace("chorus", "").lower()
        # modified_song = modified_song.replace("chorus", "").replace("&", "silencio").lower()
        # modified_song = modified_song.replace("&", "silencio").lower()
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        modified_song = regex.sub('', modified_song)
        songs.append(modified_song)
    songs.pop(305)
    return songs

def remove_words(words: List[str], songs):
    resulting_songs = []
    for song in songs:
        result = song
        for word in words:
            result = result.replace(word, '')
        resulting_songs.append(result)
    return resulting_songs

In [4]:
class LyricsGenerator(object):

    def __init__(self, embedding_dim: int, vocab_size: int, input_size: int, embedding_matrix: np.ndarray):
        embedding_layer = Embedding(
            vocab_size,
            embedding_dim,
            input_length=input_size,
            weights=[embedding_matrix],
            # embeddings_initializer=initializers.Constant(embedding_matrix),
            trainable=False,
        )

        self.model = Sequential()
        self.model.add(embedding_layer)
        self.model.add(LSTM(units=embedding_dim))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(units=vocab_size, activation='softmax'))
        self.model.compile(optimizer='adam', loss='categorical_crossentropy')
        self.model.summary()

    def fit(self, x, y, hyper_parameters):
        callback = callbacks.EarlyStopping(monitor='loss', patience=3)
        callback2 = callbacks.LearningRateScheduler(self._lr_scheduler)
        # callback3 = callbacks.ModelCheckpoint('lyrics_model.h5', save_best_only=True, monitor='val_loss', mode='min')
        # callback = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=1)
        # history = self.model.fit(x, y, epochs=100, verbose=1)
        history = self.model.fit(x, y, batch_size=hyper_parameters['batch_size'], epochs=hyper_parameters['epochs'],
                                 callbacks=[callback, callback2],
                                 verbose=1, validation_split=hyper_parameters['validation_split'],
                                 validation_data=None)
        self.model.save('lyrics_model.h5')
        
        return history

    def _lr_scheduler(self, epoch, lr):
        return 0.99 * lr

    # to get a picture of loss progress.
    def plot_metric(self, history: History, metric: str = 'loss') -> None:
        import matplotlib.pyplot as plt
        train_metrics = history.history[metric]
        val_metrics = history.history['val_'+metric]
        epochs = range(1, len(train_metrics) + 1)
        plt.plot(epochs, train_metrics)
        plt.plot(epochs, val_metrics)
        plt.title('Training and validation '+ metric)
        plt.xlabel("Epochs")
        plt.ylabel(metric)
        plt.legend(["train_"+metric, 'val_'+metric])
        plt.show()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
embedding_matrixs =  np.load("/content/drive/MyDrive/DL/Assignment_3/Lyrics/embedding_matrix_glove.npy")
embedding_matrixs.shape

(7527, 300)

In [7]:
songs = load_songs('/content/drive/MyDrive/DL/Assignment_3/Lyrics/lyrics_train_set.csv')

# Data preprocessing
tokenize = Tokenizer()
tokenize.fit_on_texts(songs)
total_words = len(tokenize.word_index) + 1

In [8]:
# import tensorflow as tf
# import random
# tf.random.set_seed(1)
# random.seed(2)
# np.random.seed(3)
import json
non_words_dict = json.load(open('/content/drive/MyDrive/DL/Assignment_3/Lyrics/non_words.json', "r"))
non_words_list = non_words_dict["non existing words"]
cleaned_songs = remove_words(non_words_list, songs)
print(len(cleaned_songs))


599


In [9]:
# # Data preprocessing
# tokenize = Tokenizer()
# tokenize.fit_on_texts(cleaned_songs)
# total_words = len(tokenize.word_index) + 1

embedding_dim = 300
x_train, y_train, max_sequence_len = create_x_y_train(cleaned_songs, tokenize, total_words)

[[221, 2451], [221, 2451, 2452], [221, 2451, 2452, 314], [221, 2451, 2452, 314, 3], [221, 2451, 2452, 314, 3, 47], [221, 2451, 2452, 314, 3, 47, 212], [221, 2451, 2452, 314, 3, 47, 212, 1], [221, 2451, 2452, 314, 3, 47, 212, 1, 67], [221, 2451, 2452, 314, 3, 47, 212, 1, 67, 18], [221, 2451, 2452, 314, 3, 47, 212, 1, 67, 18, 1]]


In [ ]:
# Train model

parameters = {
    'batch_size' : 4 ,
    'validation_split' : None ,
    'epochs' : 4 ,
    'val_data' : None
}
lyrics_generator = LyricsGenerator(embedding_dim, total_words, x_train.shape[1], embedding_matrixs)
h = lyrics_generator.fit(x_train, y_train, parameters)
# lyrics_generator.plot_metric(h)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 99, 300)           2258100   
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 7527)              2265627   
Total params: 5,244,927
Trainable params: 2,986,827
Non-trainable params: 2,258,100
_________________________________________________________________
Epoch 1/4
38234/38234 [==============================] - 289s 7ms/step - loss: 5.9240
Epoch 2/4
38234/38234 [==============================] - 287s 8ms/step - loss: 4.9248
Epoch 3/4
38234/38234 [==============================] - 283s 7m

In [ ]:
def generate_text(seed_text, eof, model, max_sequence_len, vocab_size):
    next_word = ""
    word_indices = np.arange(vocab_size) + 1
    while next_word != eof:
        token_list = tokenize.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_proba = model.model.predict_proba(token_list, verbose=0)
        # print(predicted_proba.reshape((predicted_proba.shape[1])).shape)
        # sample the word index acording to predicted proba
        chosen_index = np.random.choice(word_indices ,1 ,replace=False, p=predicted_proba.reshape((predicted_proba.shape[1])))

        
        next_word = ""
        for word, index in tokenize.word_index.items():
            if index == chosen_index:     
                next_word = word
                print(next_word)
                break
        seed_text += " " + next_word
    return seed_text

In [ ]:
print(generate_text('forgive' , "eof" , lyrics_generator , max_sequence_len, total_words))